# Guidelines for Prompting
[video link](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)

各種openaiライブラリの使用方法とそのAPIの使い方などのガイドライン



### 個人メモ

pipenv での仮想環境構築方法

    export PIPENV_VENV_IN_PROJECT=true

    pipenv sync

## Open AIライブラリのインストール

もしOpenAIのライブラリをインストールしていなければpip(またはpipenv)でインストール

In [26]:

#!pip install openai

## API Key
OpenAIのAPI Keyが必要なため、それをセット

取得方法は以下の通り

[このリンク](https://platform.openai.com/account/api-keys)にアクセスし、API Keyを取得。

もしOpenAIのアカウントが無ければ、Googleアカウントで簡単に作れるので作成してください。

取得したAPI Keyはapi_key.txtというファイル内にコピーしてもらえれば、下のセルで自動的に読み込みに行きます。


In [27]:
f = open("api_key.txt", "r")
api_key = f.read()
#api_key

In [28]:
import openai
openai.api_key=api_key

In [29]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())



### Helper Function

OpenAI の*gpt-3.5-turbo*を使う上で、簡単にAPIを使って出力を確認できる以下の関数を利用する

In [30]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"user", "content":prompt}]
    responce = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0 # modelからの出力のランダム量を調整するパラメータ
    )
    return responce.choices[0].message["content"]

### 実行時の原則

1. 明確、限定的に指示を書くこと　
2. モデルに考える時間を与えること

### Principle1(原則1): 明確、限定的に指示を書くこと

#### Tactics1: Delimiter(区切り文字)を使い、入力部を区別できるようにする。

要約をお願いする場合など、最初に要約をしてほしい文章を入力するが、その文章を**Delimiter(区切り文字)**を使ってその後の指令と区別できるようにする。

Delimiterには以下の物がある。

- """ : Triple Quotes
- \`\`\` : Triple Backtics
- \-\-\- : Triple Dashes
- < > : Angle Brackets
- \<tag> \</tag> : XML tags

In [31]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

In [32]:
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

responce = get_completion(prompt)
#print(responce)
responce

'Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.'

#### Tactics2 : 構造体での出力をリクエストする

JSON, HTMLでの出力もリクエストできる

In [33]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Life of Bees",
    "author": "Lila Rose",
    "genre": "Romance"
  }
]


#### Tactics3 : モデルにリクエストした条件が達成されているか確認してもらう

In [34]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


In [35]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-1MwcbnzTUepoLZo4Jxidl1UK on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

#### Tactics4 : "Few-Shot" Prompting

モデルにタスクをやらせる前に、そのタスクの正解例を与えてあげること。(教師あり学習的なイメージか？)

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

### Principle2(原則2) : モデルに考える時間を与える

複雑な問題をすぐに解かせようとすると間違った回答に行きついてしまう。

なので、タスクに対しより多くの計算時間を与えるようにリクエストすることでこのような事態を避けることができる。

#### Tactics1 : タスクを達成するためにステップを刻むように指示する

下のように手順を分割してリクエストする。

Step 1: ...
Step 2: ...
...
Step N: ...

恐らくstep-by-stepで、という指令を入れた方が上手く行く、と言われているのもここに当たると思われる


In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

出力方法を以下のように指示するとより出力が安定する

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

#### Tactics2 : モデルに結論を急がずに自らの解決策を考えるように指示する

以下に例を示す

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

この例だと、生徒の解決法は正しい、と判定されているが、実際は間違っている。(最後の100xが10x)

なので、こちらの解決法を書いた後に、GPT側に**別途解決法を考えてもらい**結果を出すように指令を出すことで望んだ結果を得られるようになる。

以下がその例

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

### Model Limitations モデルの制限 

#### Hallucination(幻覚、幻影)

もっともらしい正しくない結果を出す現象を**Hallucination**と呼ぶ


#### Hallucinationを防ぐには?

- 関係する情報を見つける
- 関係する情報をもとに質問に回答させる


In [ ]:
# Hallucination発生

prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)